# **CS349F Review Session 1**

# Table of Contents
* [**CS349F Review Session 1**](#**CS349F-Review-Session-1**)
* [Setup](#Setup)
* [Utility Functions](#Utility-Functions)
* [0. CloudEx Data Types](#0.-CloudEx-Data-Types)
	* [Order](#Order)
	* [Trade](#Trade)
* [1. Submitting Orders](#1.-Submitting-Orders)
	* [1.1 Setup CloudEx Trader](#1.1-Setup-CloudEx-Trader)
	* [1.2 Submitting a Buy Order](#1.2-Submitting-a-Buy-Order)
	* [1.3 Submitting a Sell Order](#1.3-Submitting-a-Sell-Order)
	* [1.4 View Pending Orders](#1.4-View-Pending-Orders)
	* [1.5 Cancelling an Order](#1.5-Cancelling-an-Order)
* [2. Fetching Real-time Market Data](#2.-Fetching-Real-time-Market-Data)
	* [2.1 Subscribe to data for a set of Symbols](#2.1-Subscribe-to-data-for-a-set-of-Symbols)
	* [2.2 Fetch Recent Trades](#2.2-Fetch-Recent-Trades)
	* [2.3 Fetch Recent Limit Order Books](#2.3-Fetch-Recent-Limit-Order-Books)
	* [2.4 Fetching Portfolio Matrix](#2.4-Fetching-Portfolio-Matrix)
* [3. Fetching Historical Symbol Market Data](#3.-Fetching-Historical-Symbol-Market-Data)
* [4. Fetching Historical Personal Data](#4.-Fetching-Historical-Personal-Data)
	* [4.1 Fetching Historical Personal Orders](#4.1-Fetching-Historical-Personal-Orders)
	* [4.2 Fetch Historical Personal Trades](#4.2-Fetch-Historical-Personal-Trades)


# Setup

In [ ]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import redis

# CloudEx imports.
import cloud_ex

# Start Redis and its Python API.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEX and VM-specific config 
def get_vm_config():
    with open("vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()

# Utility Functions

In [ ]:
ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order IDs to outstanding orders, and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

# 0. CloudEx Data Types

## Order

In [ ]:
dummy_order = "AA|AA.099950160218272693456410000063|NULL|C3|L|B|1602182726927431|1602182726934577|1602182726934784|1602182726934928|7|50|V|100"
order_object = cloud_ex.Order(dummy_order)

# These are the fields contained in an Order object
print(ORDER_FIELDS_LIST)

In [ ]:
print(order_object.symbol_)
print(order_object.order_id_) # assigned by gateway
print(order_object.type_)
print(order_object.action_)
print(order_object.limit_price_)
print(order_object.result_) # whether order was valid/invalid; assigned by matching engine
print(order_object.num_shares_)

## Trade

In [ ]:
dummy_trade = "AA|4|5|AA.099950160218241778214310000033|AA.000050160218241778354510000043|C3|C3|50|5000|100|1602182417783908|1602182417784258|10003"
trade_object = cloud_ex.Trade(dummy_trade)

# These are the fields contained in an Trade object
print(TRADE_FIELDS_LIST)

In [ ]:
print(trade_object.symbol_)
print(trade_object.exec_price_)
print(trade_object.cash_traded_)
print(trade_object.shares_traded_) # whether order was valid/invalid; assigned by matching engine
print(trade_object.trade_serial_num_)

# 1. Submitting Orders

## 1.1 Setup CloudEx Trader

This object will instantiate a connection to a gateway in CloudEx. This Gateway will relay order to the matching engine and market data to your client.

**Arguments to `Trader` constructor:**
- gateway_ip `str` - The IP address for the gateway assigned to you
- client_id `str` - Your client identifier
- client_token `str` - Your client token

In [ ]:
# Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
gateway_ip = config["gateway_ip"]
client_id = config["client_id"]
client_token = config["client_token"]

# Clear any existing data locally.
redis_api = redis.Redis()
redis_api.flushall();

# Create trader object.
trader = cloud_ex.Trader(gateway_ip, client_id, client_token)

In [ ]:
# Get a list of all symbols available for trading.
symbol_list = trader.GetSymbols()
print("These are all the {} symbols available for trading at the CloudEx exchange: {}".format(
    len(symbol_list),symbol_list))

## 1.2 Submitting a Buy Order

We will create a buy order object and collect information on the order we would like to place. 

**Arguments to `Trader.SubmitOrder()`:**
- symbol `str` - The symbol we would like to trade
- order `cloud_ex.Order` - The Order object that will be updated by submission info
- order_type `cloud_ex.OrderType` - The type of order to place, whether a limit (`cloud_ex.OrderType.limit`) or a market order (`cloud_ex.OrderType.market`)
- order_action `cloud_ex.OrderType` - Whether the order is a buy (`cloud_ex.OrderAction.buy`) or sell order (`cloud_ex.OrderAction.sell`)
- num_shares `int` - Number of shares to buy/sell
- price `int` - The order's price (integer) 

In [ ]:
# Create a limit buy order.
order_ = cloud_ex.Order() # Create an order object.
type_ = cloud_ex.OrderType.limit # Limit Order
action_ = cloud_ex.OrderAction.buy # Mark the order as a buy.

# Fill in details on what symbol you will buy, how many shares and for how much.
symbol = 'AA'
num_shares = 100
price = 45

# Place the order.
result = trader.SubmitOrder(symbol, order_, type_,
                                  action_, num_shares, price)

In [ ]:
# Let's check the order ID now that we've submitted the order.
if result == cloud_ex.OrderResult.in_sequencer:
  print(order_.order_id_) # Our order is at the matching engine.
else:
    print("ERROR: return code: {}".format(result))

## 1.3 Submitting a Sell Order


We will place an order as before but we set the order action to `cloud_ex.OrderAction.sell`.

In [ ]:
# Create a limit sell order.
order_ = cloud_ex.Order() # Create an order object.
type_ = cloud_ex.OrderType.limit # Limit Order
action_ = cloud_ex.OrderAction.sell # Mark the order as a sell.

# Fill in details on what symbol you will buy, how many shares and for how much.
symbol = 'AA'
num_shares = 80
price = 50

# Place the order.
result = trader.SubmitOrder(symbol, order_, type_,
                                  action_, num_shares, price)

In [ ]:
# Again, let's check the order ID now that we've submitted the order.
if result == cloud_ex.OrderResult.in_sequencer:
  order_id = order_.order_id_
  print(order_id) # Our order is at the matching engine.
else:
    print("ERROR: return code: {}".format(result))

## 1.4 View Pending Orders

You can view all outstanding orders (those that have not been fulfilled) by calling the **`Trader.GetOutstandingOrders`** function. Doing so can give you the information needed to cancel any or all orders.

In [ ]:
outstanding_orders = cloud_ex.MapStringOrder()

trader.GetOutstandingOrders(outstanding_orders)
print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders

## 1.5 Cancelling an Order

Given an order ID, we can cancel it if has not been fulfilled. <br/>
Let's try to cancel the sell order we just placed using the **`Trader.SubmitCancel`** function.

**Arguments to `Trader.SubmitCancel()`:**
- order_id `str` - The order id for order to cancel.

In [ ]:
result = trader.SubmitCancel(order_id)

# Let's check to see the status of the cancelled order.
if result == cloud_ex.OrderResult.in_sequencer:
  print("Cancel order succesfully reaches the matching engine.")
else:
  print("ERROR: return code: {}".format(result))

# 2. Fetching Real-time Market Data

## 2.1 Subscribe to data for a set of Symbols

Before we fetch market data, we will configure our trading api to pull data for a subset of tickers we are interested in.

Arguments to **`Trader.ConfigActiveSymbols`**:
- symbols `list` - list of tickers `str` that we will fetch data for

In [ ]:
active_symbol_list = ['AA', 'AB', 'AC', 'AD', 'AE']

success = trader.ConfigActiveSymbols(active_symbol_list)
if not success:
    print("Error setting active symbols. Check that all symbols in {} are in the full symbol list.".format(active_symbol_list))
else:
    print("Successfully subscribed to {}".format(active_symbol_list))

## 2.2 Fetch Recent Trades

For any of the symbols we have subscribed to, we can pull recent trades. We will use the **`Trader.GetRecentTrades`** to do so.

Arguments to **`Trader.GetRecentTrades()`**:
- symbol `str` - Ticker we will fetch data for.
- trade_vec `cloud_ex.VectorTrade` - Data structure to hold the recent trades.
- start_fetch_time `int` - Time (in unix epoch microseconds) to start fetching data from.

In [ ]:
symbol = 'AA'
past_seconds = 60
trade_vec = cloud_ex.VectorTrade()
start_fetch_time_us = int((time.time() - past_seconds) * 1e6) # We will fetch from past_seconds seconds ago to now.

success = trader.GetRecentTrades(symbol, trade_vec, start_fetch_time_us)
if not success:
    print("Error getting {symbol} trades. Check that {symbol} is in your active symbol list".format(symbol=symbol))
else:
    print("There were {} Trades for {} in the last {} seconds.".format(len(trade_vec), symbol, past_seconds))

# Transform recent trades into a DataFrame
recent_trade_df = TradeDF(trade_vec)
recent_trade_df

## 2.3 Fetch Recent Limit Order Books

For any of the symbols we have subscribed to, we can pull recent limit order books. We will use the **`Trader.GetRecentLOBs`** to do so.

Arguments to **`Trader.GetRecentLOBs()`**:
- symbol `str` - Ticker we will fetch data for.
- trade_vec `cloud_ex.VectorLOB` - Data structure to hold the recent LOBs.
- start_fetch_time `int` - Time (in unix epoch microseconds) to start fetching data from.

In [ ]:
symbol = 'AA'
past_seconds = 60
lob_vec = cloud_ex.VectorLOB()
start_fetch_time_us = int((time.time()-past_seconds) * 1e6) # We will fetch from 5 seconds ago to now.

success = trader.GetRecentLOBs(symbol, lob_vec, start_fetch_time_us)
if not success:
    print("Error getting {symbol} LOBs. Check that {symbol} is in your active symbol list".format(symbol=symbol))
else:
    print("Received {} LOBs for {} in the last {} seconds.".format(len(lob_vec), symbol, past_seconds))

In [ ]:
import datetime
for lob in lob_vec:
    print(datetime.datetime.fromtimestamp(10**-6*lob.creation_timestamp_))

## 2.4 Fetching Portfolio Matrix

We can track the current portfolio matrix using the **`Trader.GetPortfolioMatrix`** function. It populates a MapStringInt with our portfolio values, mapping from symbol name (`str`) to symbol balance (`int`). 

Note that your cash is denoted as `CASH` in this map.

In [ ]:
portfolio_mat = cloud_ex.MapStringInt()

trader.GetPortfolioMatrix(portfolio_mat)

print(portfolio_mat)

# 3. Fetching Historical Symbol Market Data

In [ ]:
symbol = 'AA'
seconds_in_past = 60*60*24 # One day
end_time_ms = int(time.time()*1e3)
start_time_ms = end_time_ms - int(seconds_in_past*1e3)

symbol_trades_vec = cloud_ex.VectorTrade()

cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
print("There are a total of {} trade(s) in the last day.".format(len(symbol_trades_vec)))
symbol_historical_trades_df = TradeDF(symbol_trades_vec)

symbol_historical_trades_df

# 4. Fetching Historical Personal Data

## 4.1 Fetching Historical Personal Orders

You can view all your historical orders using **`Trader.GetAllHistoricalOrders`**. The resulting data can inform which orders you placed and when.

In [ ]:
my_historical_orders = cloud_ex.VectorOrder()

trader.GetAllHistoricalOrders(my_historical_orders)
print("You have submitted a total of {} order(s).".format(len(my_historical_orders)))
my_historical_orders_df = OrderDF(my_historical_orders)

my_historical_orders_df

In [ ]:
my_historical_orders_df[my_historical_orders_df['ResultType']!='V']

## 4.2 Fetch Historical Personal Trades

You can view all your historical trades using **`Trader.GetAllHistoricalTrades`**. The resulting data can inform which of your orders got transacted and for how much.

In [ ]:
my_historical_trades = cloud_ex.VectorTrade()

trader.GetAllHistoricalTrades(my_historical_trades)
print("You have made a total of {} trade(s).".format(len(my_historical_trades)))
my_historical_trades_df = TradeDF(my_historical_trades)

my_historical_trades_df